**This is just an introduction to the new ds4n6_lib, showing some of the analysis we can perform with just a few instructions. During the next weeks we will be publishing different blog posts explaining new techniques and commands to get more powerful analysis**

## **FUNCTIONS, VARS, INIT**

**We create a couple of variables we will use later: home_path (home path of the created environment) and thisnb (name of the current notebook).**

In [1]:
home_path=!pwd
thisnb=home_path[0]+"/szechuan_volatility.ipynb"

## **IMPORTS & INIT**

**We load all the needed libraries defined in the imports.ipynb notebook and deactivate the warnings in order to have a clean result.**

In [2]:
%run imports.ipynb

## **READ DATA**

**We download the outputs we got from running volatility in the szechuan evidence.**

In [3]:
url = 'https://drive.google.com/uc?id=1nRxNTpPuZTzvydCwjfxIhtGa5OdaLDFw'
of = 'volatility.tgz'
gdown.download(url, of, quiet=False)
gdown.extractall("volatility.tgz", to="volatility/")

Downloading...
From: https://drive.google.com/uc?id=1nRxNTpPuZTzvydCwjfxIhtGa5OdaLDFw
To: /home/jovyan/volatility.tgz
15.2MB [00:00, 38.3MB/s]


['volatility/vol-wndscan.out.txt.psv',
 'volatility/vol-wintree.out.txt.psv',
 'volatility/vol-windows.out.txt.psv',
 'volatility/vol-vmwareinfo.out.txt.psv',
 'volatility/vol-vboxinfo.out.txt.psv',
 'volatility/vol-vadwalk.out.txt.psv',
 'volatility/vol-vadtree.out.txt.psv',
 'volatility/vol-vadinfo.out.txt.psv',
 'volatility/vol-usnparser.out.txt.psv',
 'volatility/vol-userhandles.out.txt.psv',
 'volatility/vol-userassist.out.txt.psv',
 'volatility/vol-unloadedmodules.out.txt.psv',
 'volatility/vol-truecryptsummary.out.txt.psv',
 'volatility/vol-truecryptpassphr.out.txt.psv',
 'volatility/vol-truecryptmaster.out.txt.psv',
 'volatility/vol-timers.out.txt.psv',
 'volatility/vol-timeliner.out.txt.psv',
 'volatility/vol-threads.out.txt.psv',
 'volatility/vol-thrdscan.out.txt.psv',
 'volatility/vol-symlinkscan.out.txt.psv',
 'volatility/vol-svcscan.out.txt.psv',
 'volatility/vol-ssdt.out.txt.psv',
 'volatility/vol-sockscan.out.txt.psv',
 'volatility/vol-sockets.out.txt.psv',
 'volatility/

**Let's read the volatility outputs with the xread() function from ds4n6_lib. It will generate the .pkl file we will use with the dataframes and store the collection of dataframes in d4.out**

In [10]:
%%time

d4.debug = 0

# Read raw data
xread(tool="volatility", rootpath=home_path[0]+"/volatility", prefix="vol-", notebook_file=thisnb, ext=".out.txt.psv", separator="|", drop_non_ham_cols=False, use_pickle=True)

- Searching notebook for saved input file / folder (_volatilityd)
  + Found: /home/jovyan/volatility

- Reading file(s):
  + INFO: 90 files found to read.

  + [1/90] vol-handles.out.txt.psv                                      -> handles                                 [ERROR]
  + [2/90] vol-driverscan.out.txt.psv                                   -> driverscan                              [83]
  + [3/90] vol-screenshot.out.txt.psv                                   -> screenshot                              [Empty File]
  + [4/90] vol-ldrmodules.out.txt.psv                                   -> ldrmodules                              [171]
  + [5/90] vol-windows.out.txt.psv                                      -> windows                                 [0]
  + [6/90] vol-moddump.out.txt.psv                                      -> moddump                                 [193]
  + [7/90] vol-clipboard.out.txt.psv                                    -> clipboard                            

  + [74/90] vol-mftparser.out.txt.psv                                    -> mftparser                               [80366]
  + [75/90] vol-modules.out.txt.psv                                      -> modules                                 [193]
  + [76/90] vol-userhandles.out.txt.psv                                  -> userhandles                             [Empty File]
  + [77/90] vol-gdt.out.txt.psv                                          -> gdt                                     [Empty File]
  + [78/90] vol-shimcache.out.txt.psv                                    -> shimcache                               [0]
  + [79/90] vol-iehistory.out.txt.psv                                    -> iehistory                               [Empty File]
  + [80/90] vol-hashdump.out.txt.psv                                     -> hashdump                                [1]
  + [81/90] vol-userassist.out.txt.psv                                   -> userassist                              [0]
  + [82

**We store the dataframes in a more friendly variable: voldfs.**

In [5]:
voldfs = d4.out

In [6]:
# Automatically created - DO NOT EDIT OR REMOVE unless you want to change the file to read (in that case, remove this cell)
# 2021-04-07 16:04:56.499470
# _volatilityd_f2read = "/home/jovyan/volatility"

**This is an output generated by the framework that is used to let you know the input you are reading.**

## **ANALYSIS**

**xdisplay is used to configure the settings of the default dataframe that will be displayed in the next cell.**

In [7]:
xdisplay()

#### Pandas options

IntText(value=60, description='display.max_rows: ', style=DescriptionStyle(description_width='initial'))

IntText(value=10, description='display.min_rows: ', style=DescriptionStyle(description_width='initial'))

IntText(value=20, description='display.max_columns: ', style=DescriptionStyle(description_width='initial'))

Dropdown(description='display.colheader_justify: ', options=('right', 'left'), style=DescriptionStyle(descript…

Checkbox(value=True, description='display.expand_frame_repr', indent=False, style=DescriptionStyle(description…

**We use the xmenu is to select and display the dataframe we want.** 

In [8]:
xmenu(voldfs)

**DataFrame visualization menu:**

Box(children=(Dropdown(description='Select DataFrame', layout=Layout(width='550px'), options=('Select DataFram…

Output(layout=Layout(overflow='auto', width='99%'))

Output(layout=Layout(overflow='auto', width='99%'))

**With xanalysis function you can run different predefined analysis. You can use it with a dataframe or a collection (dictionary).**

In [9]:
xanalysis(voldfs)

**Analisys explorer:**

Box(children=(Dropdown(description='Analysis object: ', layout=Layout(width='250px'), options=('Select one...'…

Box(children=(Dropdown(description='Available analysis types: ', layout=Layout(width='550px'), options=(), sty…

Output(layout=Layout(overflow='auto', width='99%'))

**This is just an introduction to the new ds4n6_lib, showing some of the analysis we can perform with just a few instructions. During the next weeks we will be publishing different blog posts explaining new techniques and commands to get more powerful analysis**